### Capstone - Battle of Neighborhoods 

### Exploring coffee shop business in Taipei ☕️
Rachel Chen @Taipei Oct, 2020

### 🔸 Introduction
__This final project explores the competitive locations for coffee shops in Taipei, Taiwan.__ With the popularity of coffee culture in Taiwan, coffee shops proliferate rapidly in Taiwan in recent years, especially in the capital - Taipei. This project aims at potential entrepreneurs who want to start a new business of opening a new coffee shop.  Therefore, we will spot the clusters of items and thus we can discover which neighborhood has a high density of coffee shops business.  Also, We will find out the best location for starting up a new coffee shop business.

_Reference:
[Taipei-Wikipedia](https://en.wikipedia.org/wiki/Taipei), [How Social Media Fuels Taiwan's Growing Café Culture](https://international.thenewslens.com/article/105387)_

### 🔸 Data
For this project, we need to collect all coffee shops data in each neighborhood of Taipei.

_Data source: [FourSquare API](https://developer.foursquare.com/)_

Description : By using __Foursquare API__ via the "request" library, we will get all the venues in each neighborhood. We can filter these venues to get only "Coffee" related items. Therefore, we can get data from all coffee shops in Taipei City including their name, address, latitudes, longitudes, and ratings. We then can pick up the hot spot area where locates most of the venues. To assess the data, we use FourSquare API and folium for visualizing a particular area and predict an appropriate location for opening a new coffee shop in Taipei.


### 🔸 Methodology
• __FourSquare API__ will be used to locate all coffee shops in Taipei. 

• Data will be sorted based on ratings.

• Finally, the data be will be visualized by using __folium__.

#### (1) We import all the libraries we need.

In [1]:
# Import library to deal with request
import requests 
# Import libraries for data analysis
import pandas as pd 
import numpy as np

# Install and import library for visualizing maps
!pip install folium
import folium 
print('Import finished!')

Import finished!


#### (2) Apply your credential ID on FourSquare.

In [21]:
CLIENT_ID = 'Your FourSquare ID' # Your FourSquare ID
CLIENT_SECRET = 'Your FourSquare Secret' # Your FourSquare Secret
VERSION = '20201028'
LIMIT = 100
print('Your credentails: ')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails: 
CLIENT_ID: Your FourSquare ID
CLIENT_SECRET: Your FourSquare Secret


Next, we can get requests near Taipei city.

In [3]:
# Get request near Taipei City

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20201028',
    "section": "coffee",
    "near": "Taipei",
    "radius": 1000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params = request_parameters)

In [4]:
data

<Response [200]>

#### (3) Transform data into json format and then we request geocode.

In [5]:
# Transform data into json format
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [6]:
print(d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"])
print('\n')                                                         
print(d["suggestedBounds"], d["totalResults"])
print('\n') 
print(d["geocode"])

city Taipei Taipei


{'ne': {'lat': 25.05519586803733, 'lng': 121.54266930512904}, 'sw': {'lat': 25.039539054156194, 'lng': 121.5209854171801}} 78


{'what': '', 'where': 'taipei', 'center': {'lat': 25.04776, 'lng': 121.53185}, 'displayString': 'Taipei, Taiwan', 'cc': 'TW', 'geometry': {'bounds': {'ne': {'lat': 25.210402914205286, 'lng': 121.6659869734425}, 'sw': {'lat': 24.960319022116305, 'lng': 121.45875100000019}}}, 'slug': 'taipei', 'longId': '72057594039596277'}


#### (4) Then we start creating groups including recommended information.

In [7]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [8]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

#### (5) Creat feature items of coffee shops and their attributes such as id, address, name...

In [19]:
items = d["groups"][0]["items"]
print("Number of items: %i" % len(items))
items[0] # Test

Number of items: 78


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '54aaddea498e23ff9855c494',
  'name': 'KiOSK',
  'location': {'address': '新生北路一段40號',
   'lat': 25.046284,
   'lng': 121.531488,
   'labeledLatLngs': [{'label': 'display',
     'lat': 25.046284,
     'lng': 121.531488}],
   'postalCode': '104',
   'cc': 'TW',
   'city': '台北市',
   'country': '臺灣',
   'formattedAddress': ['新生北路一段40號', '台北市,  104', '臺灣']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-54aaddea498e23ff9855c494-0'}

We can identify necessary factors from above dict for what we will use later to consider the probability of launching  location.

In [10]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i coffee shops" % len(df))
df.head()

{'address': '新生北路一段40號', 'lat': 25.046284, 'lng': 121.531488, 'labeledLatLngs': [{'label': 'display', 'lat': 25.046284, 'lng': 121.531488}], 'postalCode': '104', 'cc': 'TW', 'city': '台北市', 'country': '臺灣', 'formattedAddress': ['新生北路一段40號', '台北市,  104', '臺灣']}
{'address': '伊通街19巷8號', 'lat': 25.04881463348538, 'lng': 121.53499910429977, 'labeledLatLngs': [{'label': 'display', 'lat': 25.04881463348538, 'lng': 121.53499910429977}], 'postalCode': '104', 'cc': 'TW', 'city': '中山區', 'state': '臺北市', 'country': '臺灣', 'formattedAddress': ['伊通街19巷8號', '中山區, 臺北市 104', '臺灣']}
{'address': '新生北路一段15號', 'lat': 25.046633, 'lng': 121.531849, 'labeledLatLngs': [{'label': 'display', 'lat': 25.046633, 'lng': 121.531849}], 'cc': 'TW', 'neighborhood': '中山區', 'city': '台北市', 'country': '臺灣', 'formattedAddress': ['新生北路一段15號', '台北市', '臺灣']}
{'address': '松江路93巷2號', 'lat': 25.050519528207236, 'lng': 121.53361336920035, 'labeledLatLngs': [{'label': 'display', 'lat': 25.050519528207236, 'lng': 121.53361336920035}], '

,uid,name,shortname,address,postalcode,lat,lng
0,54aaddea498e23ff9855c494,KiOSK,Café,,104,25.046284,121.531488
1,53788c7d498e4403b48271b6,沐鴉咖啡,Coffee Shop,,104,25.048815,121.534999
2,4ce7cc4db99754816373f844,93巷人文空間,Café,,104,25.050520,121.533613
3,4dc6a209c65b89d3ca8be21a,三重奏 Trio Café,Cocktail,,100,25.043830,121.528880
4,4b88a745f964a520f30632e3,CoffeeLab (咖啡實驗室),Café,,100,25.042944,121.528568


#### (6) Finally, we can get the coordinate of Taipei and create a folium map to visualize data.
Please be noted that the maps may not be shown completely in Github. Please go to "README" file for maps. 😉

In [23]:
Taipei_center = d["geocode"]["center"]
print('Coordinate of Taipei: ', Taipei_center)

Coordinate of Taipei:  {'lat': 25.04776, 'lng': 121.53185}


In [12]:
from folium import plugins

map_Taipei = folium.Map(location=[25.04776, 121.53185], zoom_start = 15)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html = True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius = 5,
            popup = label,
            color = '#ed5c5c',
            fill = True,
            fill_color = '#de9c6a',
            fill_opacity = 0.6,
            parse_html = False).add_to(map_Taipei)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Taipei.add_child(plugins.HeatMap(hm_data))

map_Taipei

We can tell which neighborhood has a high density of coffee shops from above map.

Next, we still need to identify the location for starting a new cofee shop. This location should meet below requirements:

(1) Locate at the hot spot area that we show in previous map

(2) Keep in a certain distance from existing coffee shops to avoid the competivity of business

In [18]:
lat = 25.04776
lng = 121.53185

map_Taipei = folium.Map(location=[lat, lng], zoom_start = 15)
add_markers(df)


folium.CircleMarker(
    [lat, lng],
    radius = 18,
    popup = 'New coffee shops location',
    color = '#f230e9',
    fill = True,
    fill_color = '#fff533',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_Taipei)

map_Taipei

Here we find out the best location for starting a new coffee shop (yellow spot in the map). 😃